In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "05/15"
pmc = "08003005" # 店舗ID
SPREADSHEET_KEY = '1JBqd8L9IBlCGO4nrzFAZYU8Qj2WWNatv3Ka84T_31vM' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1001 # スロットの1台目
wait = 0.01

user_id = "gar2.den8.23+005@gmail.com"
user_pw = "jol6teon23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p2row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s5table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "111":
                    p1table.append(li)
                if unit == "22":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "111":
                        p1table.append(li)
                    if unit == "22":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "555":
                    s5table.append(li)
                if unit == "222":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "555":
                        s5table.append(li)
                    if unit == "222":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['05/15', '025626', '1', '0', '0', '--', '--']
['05/15', '025841', '2', '0', '0', '--', '--']
['05/15', '025855', '3', '0', '0', '--', '--']
['05/15', '025308', '5', '0', '0', '--', '--']
['05/15', '025846', '6', '0', '0', '--', '--']
['05/15', '025698', '7', '0', '0', '--', '--']
['05/15', '025848', '8', '0', '0', '--', '--']
['05/15', '025774', '10', '0', '0', '--', '--']
['05/15', '025698', '11', '0', '0', '--', '--']
['05/15', '025855', '12', '0', '0', '--', '--']
['05/15', '025633', '13', '0', '0', '--', '--']
['05/15', '025835', '15', '0', '0', '--', '--']
['05/15', '025180', '16', '0', '0', '--', '--']
['05/15', '025563', '17', '0', '0', '--', '--']
['05/15', '025831', '18', '0', '0', '--', '--']
['05/15', '025784', '20', '0', '0', '--', '--']
['05/15', '025817', '21', '0', '0', '--', '--']
['05/15', '025605', '22', '0', '0', '--', '--']
['05/15', '025533', '23', '0', '0', '--', '--']
['05/15', '025605', '25', '0', '0', '--', '--']
['05/15', '025521', '26', '0', '0', '--', '--']

['05/15', '025579', '262', '0', '0', '--', '--']
['05/15', '025579', '263', '0', '0', '--', '--']
['05/15', '025579', '265', '0', '0', '--', '--']
['05/15', '025579', '266', '0', '0', '--', '--']
['05/15', '025579', '267', '0', '0', '--', '--']
['05/15', '025579', '268', '0', '0', '--', '--']
['05/15', '025579', '270', '0', '0', '--', '--']
['05/15', '025856', '271', '0', '0', '--', '--']
['05/15', '025856', '272', '0', '0', '--', '--']
['05/15', '025856', '273', '0', '0', '--', '--']
['05/15', '025856', '275', '0', '0', '--', '--']
['05/15', '025856', '276', '0', '0', '--', '--']
['05/15', '025856', '277', '0', '0', '--', '--']
['05/15', '025856', '278', '0', '0', '--', '--']
['05/15', '025856', '280', '0', '0', '--', '--']
['05/15', '025856', '281', '0', '0', '--', '--']
['05/15', '025856', '282', '0', '0', '--', '--']
['05/15', '025856', '283', '0', '0', '--', '--']
['05/15', '025856', '285', '0', '0', '--', '--']
['05/15', '025856', '286', '0', '0', '--', '--']
['05/15', '025856', 

['05/15', '025773', '622', '0', '0', '--', '--']
['05/15', '025802', '623', '0', '0', '--', '--']
['05/15', '025355', '625', '0', '0', '--', '--']
['05/15', '025753', '626', '0', '0', '--', '--']
['05/15', '025792', '627', '0', '0', '--', '--']
['05/15', '025436', '628', '0', '0', '--', '--']
['05/15', '025603', '630', '0', '0', '--', '--']
['05/15', '025693', '631', '0', '0', '--', '--']
['05/15', '025364', '632', '0', '0', '--', '--']
['05/15', '025801', '633', '0', '0', '--', '--']
['05/15', '025579', '635', '0', '0', '--', '--']
['05/15', '025779', '636', '0', '0', '--', '--']
['05/15', '025579', '637', '0', '0', '--', '--']
['05/15', '025604', '638', '0', '0', '--', '--']
['05/15', '025836', '650', '0', '0', '--', '--']
['05/15', '025599', '651', '0', '0', '--', '--']
['05/15', '025625', '652', '0', '0', '--', '--']
['05/15', '025750', '653', '0', '0', '--', '--']
['05/15', '025328', '655', '0', '0', '--', '--']
['05/15', '025483', '656', '0', '0', '--', '--']
['05/15', '025840', 

['05/15', '120122', '1021', '--', '0', '--', '--', '--']
['05/15', '120122', '1022', '--', '0', '--', '--', '--']
['05/15', '119984', '1023', '--', '0', '--', '--', '--']
['05/15', '119984', '1025', '--', '0', '--', '--', '--']
['05/15', '119984', '1026', '--', '0', '--', '--', '--']
['05/15', '119984', '1027', '--', '0', '--', '--', '--']
['05/15', '119984', '1028', '--', '0', '--', '--', '--']
['05/15', '120036', '1030', '--', '0', '--', '--', '--']
['05/15', '120036', '1031', '--', '0', '--', '--', '--']
['05/15', '120036', '1032', '--', '0', '--', '--', '--']
['05/15', '120036', '1033', '--', '0', '--', '--', '--']
['05/15', '120036', '1035', '--', '0', '--', '--', '--']
['05/15', '120068', '1036', '--', '0', '--', '--', '--']
['05/15', '120068', '1037', '--', '0', '--', '--', '--']
['05/15', '120068', '1038', '--', '0', '--', '--', '--']
['05/15', '120068', '1050', '--', '0', '--', '--', '--']
['05/15', '120068', '1051', '--', '0', '--', '--', '--']
['05/15', '120021', '1052', '--

['05/15', '120073', '1251', '--', '0', '--', '--', '--']
['05/15', '120073', '1252', '--', '0', '--', '--', '--']
['05/15', '120073', '1253', '--', '0', '--', '--', '--']
['05/15', '120073', '1255', '--', '0', '--', '--', '--']
['05/15', '120073', '1256', '--', '0', '--', '--', '--']
['05/15', '120073', '1257', '--', '0', '--', '--', '--']
['05/15', '120077', '1258', '--', '0', '--', '--', '--']
['05/15', '120077', '1260', '--', '0', '--', '--', '--']
['05/15', '120077', '1261', '--', '0', '--', '--', '--']
['05/15', '120077', '1262', '--', '0', '--', '--', '--']
['05/15', '120077', '1263', '--', '0', '--', '--', '--']
['05/15', '120077', '1265', '--', '0', '--', '--', '--']
['05/15', '120077', '1266', '--', '0', '--', '--', '--']
['05/15', '120077', '1267', '--', '0', '--', '--', '--']
['05/15', '120077', '1268', '--', '0', '--', '--', '--']
['05/15', '120093', '1270', '--', '0', '--', '--', '--']
['05/15', '120093', '1271', '--', '0', '--', '--', '--']
['05/15', '120093', '1272', '--

['05/15', '120068', '1571', '--', '0', '--', '--', '--']
['05/15', '120025', '1572', '--', '0', '--', '--', '--']
['05/15', '120025', '1573', '--', '0', '--', '--', '--']
['05/15', '120068', '1575', '--', '0', '--', '--', '--']
['05/15', '120068', '1576', '--', '0', '--', '--', '--']
['05/15', '119941', '1577', '--', '0', '--', '--', '--']
['05/15', '119913', '1578', '--', '0', '--', '--', '--']
['05/15', '120068', '1580', '--', '0', '--', '--', '--']
['05/15', '120025', '1581', '--', '0', '--', '--', '--']
['05/15', '120068', '1582', '--', '0', '--', '--', '--']
['05/15', '119941', '1583', '--', '0', '--', '--', '--']
['05/15', '120068', '1585', '--', '0', '--', '--', '--']
['05/15', '120025', '1586', '--', '0', '--', '--', '--']
['05/15', '120025', '1587', '--', '0', '--', '--', '--']
['05/15', '120104', '1588', '--', '0', '--', '--', '--']
['05/15', '120104', '1600', '--', '0', '--', '--', '--']
['05/15', '120104', '1601', '--', '0', '--', '--', '--']


{'spreadsheetId': '1JBqd8L9IBlCGO4nrzFAZYU8Qj2WWNatv3Ka84T_31vM',
 'updates': {'spreadsheetId': '1JBqd8L9IBlCGO4nrzFAZYU8Qj2WWNatv3Ka84T_31vM',
  'updatedRange': 'S2RAW!F3292:M3323',
  'updatedRows': 32,
  'updatedColumns': 8,
  'updatedCells': 256}}